In [6]:
import json, pickle, os, time, copy, random, base64
from collections import Counter, defaultdict
import numpy as np
import torch
from pprint import pprint

In [4]:
with open("/data/yingshac/MMMHQA/VinVL_output/x_neg_imgs_0_240661/predictions.lineidx", "r") as fp:
    pred_lineidx = [int(i.strip()) for i in fp.readlines()]
    print(len(pred_lineidx))

240700


In [7]:
i = 200001
with open("/data/yingshac/MMMHQA/VinVL_output/x_neg_imgs_0_240661/predictions.tsv", "r") as fp:
    fp.seek(pred_lineidx[i])
    row = fp.readline().strip().split('\t')
    img_id_from_pred = int(row[0])
    pred = json.loads(row[-1])['objects']
    print(img_id_from_pred)
    print("num of boxes = ", len(pred))
V = pred
pred_boxes_list = []
scores_list = []
fc1_features_list = []
cls_features_list = []
for v in V:
    pred_boxes_list.append(v['rect'])
    scores_list.append(v['conf'])
    fc1_features_list.append(np.frombuffer(base64.b64decode(v['feature']), np.float32))
    cls_features_list.append(np.frombuffer(base64.b64decode(v['scores_all']), np.float32))
pred_boxes = torch.FloatTensor(pred_boxes_list)
scores = torch.FloatTensor(scores_list)
fc1_features = torch.FloatTensor(fc1_features_list)
cls_features = torch.FloatTensor(cls_features_list)
print(pred_boxes.type())
print(scores.type())
print(fc1_features.type())
print(cls_features.type())
print(pred_boxes.size())
print(scores.size())
print(fc1_features.size())
print(cls_features.size())

199967
num of boxes =  100
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.FloatTensor
torch.Size([100, 4])
torch.Size([100])
torch.Size([100, 2048])
torch.Size([100, 1595])


In [81]:
len(os.listdir("/data/yingshac/MMMHQA/imgFeatures_upd/gold/"))

22265

In [83]:
fp = open("/data/yingshac/MMMHQA/detectron_output/gold.tsv", "w")
fp_lineidx = open("/data/yingshac/MMMHQA/detectron_output/gold.lineidx", "w")
for i in range(22265):
    if i%1000 == 999: print(i)
    im = "0"*(8-len(str(i))) + str(i)
    if not os.path.exists("/data/yingshac/MMMHQA/imgFeatures_upd/gold/{}.pkl".format(im)): 
        print("NONE: ", im)
        fp_lineidx.write('{0}\n'.format("NONE"))
        #fp.write('{0}\n'.format('NONE'))
    features = pickle.load(open("/data/yingshac/MMMHQA/imgFeatures_upd/gold/{}.pkl".format(im), "rb"))
    #print(features.keys())
    row = {}
    row['image_size'] = features['image_size']
    row['num_instances'] = features['num_instances']
    row['pred_classes'] = base64.b64encode(features['pred_classes'].detach().cpu().numpy()).decode("utf-8")
    row['scores'] = base64.b64encode(features['scores'].detach().cpu().numpy()).decode("utf-8")
    row['pred_boxes'] = base64.b64encode(features['pred_boxes'].detach().cpu().numpy()).decode("utf-8")
    row['fc1_features'] = base64.b64encode(features['fc1_features'].detach().cpu().numpy()).decode("utf-8")
    row['cls_features'] = base64.b64encode(features['cls_features'].detach().cpu().numpy()).decode("utf-8")
    x = json.dumps(row)
    line = [i, im, x]
    v = '{0}\n'.format('\t'.join(map(str, line)))
    fp_lineidx.write('{0}\n'.format(fp.tell()))
    fp.write(v)
    
fp.close()
fp_lineidx.close()

999
1999
2999
3999
4999
5999
6999
7999
8999
9999
10999
11999
12999
13999
14999
15999
16999
17999
18999
19999
20999
21999


In [79]:
with open("/data/yingshac/MMMHQA/detectron_output/gold.lineidx", "r") as fp:
    pred_lineidx = [int(i.strip()) for i in fp.readlines()]
    print(len(pred_lineidx))

500


In [80]:
i = 411
with open("/data/yingshac/MMMHQA/detectron_output/gold.tsv", "r") as fp:
    fp.seek(pred_lineidx[i])
    row = fp.readline().strip().split('\t')
    print(int(row[0]))
    print(row[1])
    pred = json.loads(row[-1])
    print(pred['image_size'])
    print(type(pred['num_instances']))
    print(np.frombuffer(base64.b64decode(pred['pred_classes']), np.int64).shape)
    print(np.frombuffer(base64.b64decode(pred['scores']), np.float32).shape)
    print(np.frombuffer(base64.b64decode(pred['pred_boxes']), np.float32).reshape((100,4)).shape)
    print(np.frombuffer(base64.b64decode(pred['fc1_features']), np.float32).reshape((100,2048)).shape)
    print(np.frombuffer(base64.b64decode(pred['cls_features']), np.float32).reshape((100,1601)).shape)

411
00000411
[1196, 800]
<class 'int'>
(100,)
(100,)
(100, 4)
(100, 2048)
(100, 1601)


In [56]:
pred = json.loads(x)
print(type(pred['image_size']))
print(type(pred['num_instances']))
print(np.frombuffer(base64.b64decode(pred['pred_classes']), np.int64).shape)
print(np.frombuffer(base64.b64decode(pred['scores']), np.float32).shape)
print(np.frombuffer(base64.b64decode(pred['pred_boxes']), np.float32).reshape((100,4)).shape)
print(np.frombuffer(base64.b64decode(pred['fc1_features']), np.float32).reshape((100,2048)).shape)
print(np.frombuffer(base64.b64decode(pred['cls_features']), np.float32).reshape((100,1601)).shape)

<class 'list'>
<class 'int'>
(100,)
(100,)
(100, 4)
(100, 2048)
(100, 1601)
